In [31]:
import pandas as pd
import numpy as np
import os,sys
from Utils.utils import *
from Utils.mRS_computation import *
from Model.Simulate import *
from Model.Mortality import Mortality
from Model.RecurrentStroke import RecurrentStroke
from Model.Utilities import Costs, QALYs

p_inputs = r'C:\Users\henkvanvoorst\Dropbox\CLEOPATRA\model_inputs'
p_inputs = r'C:\Users\hvanvoorst\Dropbox\CLEOPATRA\CLEOPATRA\model_inputs'

p_mort_file = os.path.join(p_inputs, 'Forecasted_mortality.xlsx')
p_HR_file = os.path.join(p_inputs, 'Stroke_recurrence.xlsx')
p_cost_file = os.path.join(p_inputs, 'Costs.xlsx')
p_qaly_file = os.path.join(p_inputs, 'QALYs.xlsx')


M = Mortality(
              p_mort_file = p_mort_file, 
              HR_mrs = np.array([1.54,2.17,3.18,4.55,6.55]),
              years = np.arange(2021,2030)
                )

RS = RecurrentStroke(
                    file_p_HR = p_HR_file,
                    p_mrs_postrestroke = np.array([11/233, 21/233,18/233,22/233,6/233])
                    )

C = Costs(
         file=p_cost_file,
         start_inflation=1.2, # inflation factor before simulation (2015 -> 2022)
         discounting_rate=1.04,
         inflation_rate=1.03
            )

Q = QALYs(
        file = p_qaly_file,
        discounting_rate=1.04
            )

FileNotFoundError: [Errno 2] No such file or directory: 'C:Users\\hvanvoorst\\Dropbox\\CLEOPATRA\\CLEOPATRA\\model_inputs\\Forecasted_mortality.xlsx'

In [3]:
# def compute_shift_no_evt(pt_dct, ORs):
#     """
#     pt_dct: dict with keys are also keys in ORs 
#             values are used to compute absolute shift
#     ORs: dict with keys of Tx values are Odds Ratios for favorable mRS

#     return one hot encoded probabilities of mRS for given patient
#     """
#     p_shift = 0
#     for name,OR in ORs.items():
#         if name in pt_dct.keys():
#             p_shift -= pt_dct[name]*np.log(OR)
#     mRS = np.zeros(7)
#     #compute one hot embedding
#     fl_new_mrs = pt_dct['mRS']+p_shift
#     new_mrs = int(np.floor(fl_new_mrs))
#     p_new_mrs = 1-(fl_new_mrs-new_mrs)
#     mRS[new_mrs] = p_new_mrs 
#     mRS[new_mrs+1] = 1-p_new_mrs 
#     return mRS

In [14]:
#1 model the decision
# n_patients with characteristics
np.random.seed(2)
fake_dta = np.vstack([np.arange(100),
                      np.random.randint(0,150,size=100),
                      np.random.randint(50,80,size=100),
                      np.random.randint(0,7,size=100),
                     ]).T
df = pd.DataFrame(fake_dta,columns=['ID','core_vol','age','mRS'])
df['EVT*core_vol'] = df['core_vol']/10
df['EVT'] = 1
df.index = df['ID']
    
# dictionary with ORs
ORs = {'EVT':2.52, 'EVT*core_vol':0.98}
input_dist = get_mRS_EVT_noEVT(df,ORs)
# define how patients are selected
ID_treat_select = df[df['core_vol']<=70].index

n_evt = len(ID_treat_select)
n_noevt = len(df)-len(ID_treat_select)

#compute 90d outcome
# mrs 90 day for EVT and no EVT based on CTP
mrs_ctp_evt, mrs_ctp_noevt, select_EVT, exclude_EVT = mRS_90d_2arms(input_dist,ID_treat_select)
# mrs 90 day for control arm (all EVT)
mrs_contr, __, select_EVT, __ = mRS_90d_2arms(input_dist,df.index)

#baseline vars: age, gender
mrs_ctp_noevt

noEVT_mRS_0    7.251547
noEVT_mRS_1    6.238354
noEVT_mRS_2    5.915149
noEVT_mRS_3    7.634369
noEVT_mRS_4    8.375770
noEVT_mRS_5    7.154532
noEVT_mRS_6    5.430280
dtype: float64

In [13]:
df['abs_EVT'] = df['EVT*core_vol']*np.log(ORs['EVT*core_vol'])+df['EVT']*np.log(ORs['EVT'])
df['OR_EVT'] = np.exp(df['abs_EVT'])
df['OR_noEVT'] = 1/df['OR_EVT']
df.head(3)

ID  core_vol  age  mRS  EVT*core_vol  EVT   abs_EVT    OR_EVT  OR_noEVT
0   0        34   61    0           3.4    1  0.855570  2.352714  0.425041
1   1        75   68    0           7.5    1  0.772739  2.165689  0.461747
2   2       112   55    1          11.2    1  0.697989  2.009706  0.497585

In [15]:
#(a/b)/(c/d) = (TP/FP)/(FN/TN)
#100/250 = 0.4
#1/0.01 = 0.01/xx
#100/250 = 0.4
# get data HERMES: (treatment/goodoutcome)/(notreatment/goodoutcome)
# get data HERMES: (corevol>x*EVT/goodoutcome)/(corevol>x*noEVT/goodoutcome)
# act as if notreatment is number is known (get from data)

print(100/101, 250/251, 10/11,25/26, 1/2,2.5/3.5)
print(100/250, 10/25, 1/2.5)
(.5/.5)/(.71/(1-.71))

0.9900990099009901 0.9960159362549801 0.9090909090909091 0.9615384615384616 0.5 0.7142857142857143
0.4 0.4 0.4


0.4084507042253521

In [3]:
# example for long term simulaion
SLT = SimulateLongTerm(5,M,RS,C,Q,
                 start_year = 2022)
# for PSA: SLT._probabilistic_resample()

mrs_start = np.array([0,0,.7,.2,0,.1])
patients = [[['R1','R1'],'M',70,mrs_start]]

#for arm in arms --> define patient_data with mrs_start
for data in patients:
    IDs,gender, age, mrs_start = data
    out = SLT(gender, age, mrs_start,IDs) #.5 ms --> cohort of 1000: 50 secs

In [4]:
# 7 ms per iter --> 10 minutes for 10,000 MC PSA
%timeit SLT._probabilistic_resample()
%timeit SLT(gender, age, mrs_start,IDs)

6.54 ms ± 145 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
432 µs ± 6.58 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
